# Frequentist vs Bayesianism part 2
Jake VanderPlas

http://jakevdp.github.io/blog/2014/06/06/frequentism-and-bayesianism-2-when-results-differ/

In [ ]:
try:
    import numpy as np, pandas as pd, matplotlib.pyplot as plt, emcee
except:
    ! conda install -y emcee
    import numpy as np, pandas as pd, matplotlib.pyplot as plt, emcee

## Example 1

In [ ]:
def report(method, p):
    print(f"{method} Probability of Bob Winning: {p*100:.0f}% or {(1-p)/p:.0f} to 1 against")

In [ ]:
method = "Naïve Frequentist"
p_hat = 5. / 8.
freq_prob = (1 - p_hat) ** 3
report(method, freq_prob)
# print(f"{method} Probability of Bob Winning: {p:.2f} or {(1-p)/p:.0f} to 1 against")

In [ ]:
method = "Closed-Form Bayesian"
from scipy.special import beta
bayes_prob = beta(6 + 1, 5 + 1) / beta(3 + 1, 5 + 1)
report(method, bayes_prob)

In [ ]:
method = "Simulation"
rng = np.random.default_rng(0)

# play 100000 games with randomly-drawn p, between 0 and 1
p = rng.uniform(0, 1, 100000)

# each game needs at most 11 rolls for one player to reach 6 wins
rolls = rng.uniform(0, 1, (11, len(p)))

# count the cumulative wins for Alice and Bob at each roll
Alice_count = np.cumsum(rolls < p, 0)
Bob_count = np.cumsum(rolls >= p, 0)

# sanity check: total number of wins should equal number of rolls
total_wins = Alice_count + Bob_count
assert np.all(total_wins.T == np.arange(1, 12))
print("(Sanity check passed)")

# determine number of games which meet our criterion of (A wins, B wins)=(5, 3)
# this means Bob's win count at eight rolls must equal 3
good_games = Bob_count[7] == 3
print("Number of suitable games: {0}".format(good_games.sum()))

# truncate our results to consider only these games
Alice_count = Alice_count[:, good_games]
Bob_count = Bob_count[:, good_games]

# determine which of these games Bob won.
# to win, he must reach six wins after 11 rolls.
bob_won = np.sum(Bob_count[10] == 6)
print("Number of these games Bob won: {0}".format(bob_won.sum()))

# compute the probability
sim_prob = bob_won.sum() * 1. / good_games.sum()

report(method, sim_prob)

## Example 2

In [ ]:
x = np.array([ 0,  3,  9, 14, 15, 19, 20, 21, 30, 35,
              40, 41, 42, 43, 54, 56, 67, 69, 72, 88])
y = np.array([33, 68, 34, 34, 37, 71, 37, 44, 48, 49,
              53, 49, 50, 48, 56, 60, 61, 63, 44, 71])
e = np.array([ 3.6, 3.9, 2.6, 3.4, 3.8, 3.8, 2.2, 2.1, 2.3, 3.8,
               2.2, 2.8, 3.9, 3.1, 3.4, 2.6, 3.4, 3.7, 2.0, 3.5])
plt.errorbar(x, y, e, fmt='.k', ecolor='gray');

In [ ]:
from scipy import optimize
from functools import partial

def f(theta, x=x):
    return theta[0] + theta[1] * x

def squared_loss(theta):
    dy = y - f(theta)
    return (0.5 * (dy / e) ** 2).sum()

def huber_loss(theta, c=3):
    dy = y - f(theta)
    return ((abs(dy) <  c) * (0.5 * dy ** 2) +
            (abs(dy) >= c) * (0.5 * c - abs(dy)) * -c
           ).sum()

def fit(loss):
    return optimize.fmin(loss, thetas['init'], disp=False)

def plot_fit(theta, name=None):
    plt.errorbar(x, y, e, fmt='.k', ecolor='gray')    
    xx = np.linspace(0, 100)
    plt.plot(xx, f(theta, xx), label=name)
    plt.legend()


thetas = dict()
fig, ax = plt.subplots()

nm = 'init'
t = [y.mean(), 0]
thetas[nm] = t
plot_fit(t, nm)

nm = 'L2'
t = fit(squared_loss)
thetas[nm] = t
plot_fit(t, nm)

r = [3, 10, 100]
for c in r:
    nm = f'Huber{c}'
    t = fit(partial(huber_loss, c=c))
    thetas[nm] = t
    plot_fit(t, nm)

In [ ]:
def log_prior(theta, sigma_B):
    ## uniform on [0,1]
    if (any(theta[2:] < 0) or any(theta[2:] > 1)):
        return -np.inf  # recall log(0) = -inf
    else:
        return 0

def log_normal(dy, s):
    return (np.log(2 * np.pi * s**2) + (dy / s)**2) / -2

def log_likelihood(theta, sigma_B):
    dy = y - f(theta)
    g = np.clip(theta[2:], 0, 1)  # g<0 or g>1 leads to NaNs in logarithm
    logL1 = np.log(g)   + log_normal(dy, e)
    logL2 = np.log(1-g) + log_normal(dy, sigma_B)
    return (np.logaddexp(logL1, logL2)).sum()

def log_posterior(theta, sigma_B):
    return log_prior(theta, sigma_B) + log_likelihood(theta, sigma_B)


ndim = 2 + len(x)  # number of parameters in the model
nwalkers = 50  # number of MCMC walkers
nburn = 10000  # "burn-in" period to let chains stabilize
nsteps = 15000  # number of MCMC steps to take

rng = np.random.default_rng(0)
theta_init  = rng.normal(thetas['L2'], 1.0, (nwalkers, 2))
# g_init      = rng.uniform(0.0, 1.0, (nwalkers, ndim-2))
g_init      = rng.normal (0.5, 0.1, (nwalkers, ndim-2))
params_init = np.hstack([theta_init, g_init])

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, kwargs={'sigma_B':50})
sampler.run_mcmc(params_init, nsteps)

In [ ]:
sample = sampler.chain  # shape = (nwalkers, nsteps, ndim)
sample = sample[:, nburn:, :].reshape(-1, ndim).T  # throw out first nburn steps to let chain stabilize, then reshape to (ndim, nsamples)

nm = 'Bayes'
t = sample.mean(axis=1)
thetas[nm] = t[:2]
g = t[2:]
outliers = (g < 0.5)

fig, ax = plt.subplots()
plot_fit(t, nm)
plt.plot(x[outliers], y[outliers], 'ro', ms=20, mfc='none', mec=ax.lines[-1].get_color())

In [ ]:
plt.plot(sample[0], sample[1], ',k', alpha=0.1)
plt.xlabel('intercept')
plt.ylabel('slope');

In [ ]:
fig, ax = plt.subplots()
for nm, theta in thetas.items():
    plot_fit(theta, nm)
plt.plot(x[outliers], y[outliers], 'ro', ms=20, mfc='none', mec=ax.lines[-1].get_color())
plt.title('Maximum Likelihood fit: Bayesian Marginalization');